# Why trade?

Trading only makes sense when you've got a signal. So lets write some code that searces for some!

In [1]:
import datetime as dt

import pandas as pd

from research.polygon_io import Polygon
from research.datastore import DataStore
from research.utils import (
    generate_weekday_dates, 
    get_home_directory,
    find_list_overlap
    )

poly = Polygon()
ds = DataStore(f"{get_home_directory()}/datastore")

In [2]:
ds.available_partitions.sample(1)

asset_type data_type symbol       file                 
                                     min        max count
1     EQUITY     TRADE   ALRS 2023-01-01 2023-10-07   280

# Framework for searching

My research process usually starts with a simple class I can expand. The goal is to create a reusable interface. Lets assume that we will look for a signal on a specific assist, so our class will likely go collect the data it needs for any arbitrary asset and then perform some analysis and return a result. Also, lets include a boolean whether or not this asset trades 24/7 so that we can apply this to both crypto and tradfi assets.

I'm also going to make assumptions that I'll be using trade data on equities for some of my data loader functions. Some conflicting ideas here, but we are using the code we have at the time of this writing :) (mostly equities focused code).

I'm starting with this. 
```python
class SignalExplorer:
    def __init__(
            self, 
            symbol: str, 
            datastore: DataStore,
            traditional_market_hours: bool = True
        ):
        self.symbol = symbol
        self.ds = DataStore
```

The following cell will have my final class so you can see how I expanded it to fit my analysis needs.

In [45]:
class SignalExplorer:
    def __init__(
            self, 
            symbol: str, 
            datastore: DataStore,
            traditional_market_hours: bool = True
        ):
        self.asset_type = "EQUITY"
        self.data_type = "TRADE"
        self.symbol = symbol
        self.ds = DataStore()

    # returns the state of trades data we have on this asset.
    # use this to identify if we need to go request and save additiona data.
    def get_datastore_data(self, start_date: str, end_date: str):
        expected_dates = find_list_overlap(
                pd.date_range(start_date, end_date),
                generate_weekday_dates(start_date, end_date)
        )
            
        data_state = self.ds.get_asset_datastore_state(
            asset_type =self.asset_type,
            data_type = self.data_type,
            symbols = [self.symbol],
            dates = expected_dates
        )
        missing = []
        for i in data_state.get(self.symbol):
            if i not in expected_dates:
                missing.append(i)
        if len(missing) == 0:
            return pd.concat([self.ds.load_asset_partition(
                    asset_type = "EQUITY", data_type = "TRADE", symbol = self.symbol, date = str(i)[:10]
                ) for i in expected_dates])
            
        
AR = SignalExplorer(symbol = "AR", datastore = ds)

In [46]:
AR.get_datastore_data("2023-08-01", dt.datetime.now().isoformat()[:10])

FileNotFoundError: [Errno 2] No such file or directory: '/home/pmk/datastore/EQUITY/TRADE/AR/2023-10-12.parquet'